In [1]:
"""
    Kasiski Attack using Python
    Author: Dimitrios Mamakas (f3322209)
"""

import string
from nltk import ngrams
from collections import Counter

In [2]:
# ------------------------ Function definitions ------------------------

In [3]:
"""
    Performs the decription of the Vigenère algorithm with a known key.
"""
def decrypt_vigenere(key, cipher):
    # Define our keystream
    keystream = (round(len(cipher) / len(key)) + 1) * key
    keystream = keystream[:len(cipher)]
    
    # Initial original text
    original = ''
    for x in range(0, len(cipher)):
        if string.ascii_uppercase.index(cipher[x]) - string.ascii_uppercase.index(keystream[x]) >= 0:
            letter = string.ascii_uppercase[string.ascii_uppercase.index(cipher[x]) - \
                                            string.ascii_uppercase.index(keystream[x])]
            original = original + letter
        else:
            letter = string.ascii_uppercase[len(string.ascii_uppercase) - abs(string.ascii_uppercase.index(cipher[x]) \
                                                                              - string.ascii_uppercase.index(keystream[x]))]
            original = original + letter
    
    # Return the result
    return original

In [4]:
"""
    Shifts the text (group) to a specific ammount of spaces (characters).
    
    Returns the text in the format of a list.
"""
def shift(text, amount):
    shifted = ''
    letters = string.ascii_uppercase
    for letter in text:
        shifted += letters[(letters.index(letter)-amount) % len(letters)]
    return [x for x in shifted]

In [5]:
"""
    Calculates the score (matching) given a sequence and a specific language.
"""
def calculate_score(freqs, english):
    score = 0
    for letter in string.ascii_uppercase:
        if letter in freqs:
            score = score + (freqs[letter] * english[letter])
    return score

In [6]:
# ------------------------ Kasiski Attack ------------------------

In [7]:
# Load the contents
with open('ciphertext.txt') as f:
    contents = f.readlines()

In [8]:
# Load the cipher
cipher = contents[0]

In [9]:
# Print the cipher
cipher

'CXSFBOIKLEAIBKMMCETMUBAOJSSLUFXDJHHAZBBZYXCKVPBGNHSYBOMRXCSPUXBOJHOWBUQBJBWMPXASBXIFNKAZBKBZEXBDOKZYBOOHOJGPUFKGRDFXNIQSHJVXLKMUNHFXPBQUNTOGQFBBJIHLNITNCXSKFMMBRUGBAJWQJBZRCLAHCYJXELTDBYBEVCMRDFDHEQAXBJSFJFBGQKATAPIRCXSHAIGMNWOMVSMZLJCKFYCSFUOKRMIQCETMUBJHXIDAROMZWTHARPCOYEGXQIGHVCCKNIJDQQJBBRZHBYRXAQQBJBHHJEISJBZHGEMQBFSVVBACXMVXAQQLNIOKRDWNMULVRMBSQQHAHJIMBQZHABBQHJCFVQQFJJSMUBMEOUQMBCBGJJFXFMWMBUCGGEMHATSLPBVCJDHLNKLNWEHAROAONSWXF'

In [10]:
# Define our n, for n-grams
N = 6

In [11]:
# Get the n-gram counts for given n
ngram_counts = Counter(ngrams([*cipher], N))

In [12]:
# Print the 10 most common matches for given n
ngram_counts.most_common(10)

[(('C', 'E', 'T', 'M', 'U', 'B'), 2),
 (('C', 'X', 'S', 'F', 'B', 'O'), 1),
 (('X', 'S', 'F', 'B', 'O', 'I'), 1),
 (('S', 'F', 'B', 'O', 'I', 'K'), 1),
 (('F', 'B', 'O', 'I', 'K', 'L'), 1),
 (('B', 'O', 'I', 'K', 'L', 'E'), 1),
 (('O', 'I', 'K', 'L', 'E', 'A'), 1),
 (('I', 'K', 'L', 'E', 'A', 'I'), 1),
 (('K', 'L', 'E', 'A', 'I', 'B'), 1),
 (('L', 'E', 'A', 'I', 'B', 'K'), 1)]

In [13]:
# Extract the first pattern
pattern = ''.join(ngram_counts.most_common(10)[0][0])

In [14]:
# Compute the positions
positions = {}

for pos, character in enumerate(cipher):
    # Create the specific n-gram
    n_gram = cipher[pos : pos + N]
    
    # Check if this specific n-gram belongs to our dictionary, and if not, add it
    if n_gram in positions:
        positions[n_gram].append(pos)
    else:
        positions[n_gram] = [pos]

In [15]:
# Compute the distance values
d_values = []
for dist in range(0, len(positions[pattern])-1):
    d_values.append(positions[pattern][dist+1] - positions[pattern][dist])

In [16]:
# These are the canditate keys based on our excercise scenario
canditate_keys = [4, 5, 6, 7, 8]

In [17]:
# Find the ones who divide the distance values
divisions = {
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: 0
}

# Loop through the canditate keys
for canditate_key in canditate_keys:
    for d_value in d_values:
        if d_value % canditate_key == 0:
            # This specific key, divides this specific distance value
            divisions[canditate_key] = divisions[canditate_key] + 1

# Finally, find our potential keys to try
keys = []
for k in range(0, len(list(divisions.values()))):
    if list(divisions.values())[k] != 0:
        # Drop the keys which don't divide any distance value at all
        keys.append(list(divisions.keys())[k])

In [18]:
# English letter frequencies
english_letter_freqs = {
    'A': 0.08167, 
    'B': 0.01492, 
    'C': 0.02782, 
    'D': 0.04253, 
    'E': 0.12702, 
    'F': 0.02228, 
    'G': 0.02015,
    'H': 0.06094, 
    'I': 0.06966, 
    'J': 0.00153, 
    'K': 0.00772, 
    'L': 0.04025, 
    'M': 0.02406, 
    'N': 0.06749,
    'O': 0.07507, 
    'P': 0.01929, 
    'Q': 0.00095, 
    'R': 0.05987, 
    'S': 0.06327, 
    'T': 0.09056, 
    'U': 0.02758,
    'V': 0.00978, 
    'W': 0.02360, 
    'X': 0.00150, 
    'Y': 0.01974, 
    'Z': 0.00074
}

In [19]:
# Attack to find the key
for key in keys:
    print('- Testing key size: ' + str(key))
    
    # Split into groups based on the canditate key - We need x groups
    key_length = key
    groups = []
    for group in range(0, key_length):
        # Loop through the cipher
        inner_group = []
        for letter in range(group, len(cipher), key_length):
            inner_group.append(cipher[letter])
        groups.append(inner_group)
        
    # Start with an empty key
    key_ = ''

    # Loop each group
    for group in groups:
        scores = {}

        # For each group, we need to perform exactly 25 shifts
        for shift_ in range(0, 26):
            # Group's letter frequencies
            group_freqs = {}

            # Perform a shift
            grp = shift(''.join(group), shift_)

            # For each letter of the new group
            for letter in grp:
                # Count the frequencies of this group
                group_freqs[letter] = grp.count(letter) / len(grp)

            # Calculate the score for this shift
            score = calculate_score(group_freqs, english_letter_freqs)

            scores[shift_] = score

        # Find the correct letter for this group
        letter = string.ascii_uppercase[list(scores)[list(scores.values()).index(max(list(scores.values())))]]
        key_ = key_ + letter
    
    # Prints
    print('- For key size: ' + str(key) + ' the key found was: ' + str(key_))
    print('- Decrypted message: ')
    print(decrypt_vigenere(key_, cipher))
    
    # If this is not the last element, print a double line break
    if (keys.index(key) != len(keys) - 1):
        print('\n\n')

- Testing key size: 4
- For key size: 4 the key found was: JXOT
- Decrypted message: 
TAEMSRURCHMPSNYTTHFTLEMVAVESLIJKAKTHQENGPAORMSNNEKEFSRYYOFEWLANVAKADSXCIAEITGAMZSAUMENMGSNNGVANKFNLFSRAOFMSWLIWNIGREELCZYMHECNYBEKREGECBEWANHINIALTSELFUTAERWPYIIXSIRMIXAELYTOMOTBVEVOFKSBNLMFYYUIPOVTMESMEMAINNHNMARSUYTAEORLSTEZATMVYGCMORWBOZWXARIPUXTHFTLEVOOLPHIRYGNWTHISOVPHSEHLSOMFORELVKHTVISULOSBDERTCIAETOAHUZAELOXHYXSIECMEMJOPHERTCSELARIGIUDXXCIPNZHTTHYMUTSTLORENXYMOMMTCMAMETLEYLFXCTSFNNAMREWPITSXONXHYORWESGEHJAGTSENXUNHTHIRMVEVIEW



- Testing key size: 8
- For key size: 8 the key found was: JQOTNXIZ
- Decrypted message: 
THEMORALCOMPONENTOFTHESPACESHIPEARTHMETAPHORISTHEREFORESOMEWHATPARADOXICALITCASTSHUMANSASUNGRATEFULFORGIFTSWHICHINREALITYTHEYNEVERRECEIVEDANDITCASTSALLOTHERSPECIESINMORALLYPOSITIVEROLESINLIFESUPPORTSYSTEMWITHHUMANSASTHEONLYNEGATIVEACTORSBUTWEAREPARTOFTHEBIOSPHEREANDTHESUPPOSEDLYIMMORALBEHAVIOURISIDENTICALTOWHATALLOTHERSPECIESDOWHENTIMESAREGOODEXCEPTTHATHUMANSALONETRYTOMITIGATETHEE